<a href="https://colab.research.google.com/github/Guhabrus/Laba_1_CUDA/blob/main/Laba1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update


Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


In [2]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2


--2021-09-09 15:19:28--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-09-09 15:19:28--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?bthtxavu37gB2y_iBKzjY3KPGlrFX-PVTDraM5E_d_pMz49aIq6I-FlBKtsPzjDpSul9G2pWemI90-tBCESsyMQK7Ig-_6JqCOBjgY2WPHMD4ECsu

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [43]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git


  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-kyogz8iz
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-kyogz8iz


In [5]:
%load_ext nvcc_plugin


created output directory at /content/src
Out bin /content/result.out


In [66]:
%%cu
#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include "device_functions.h"
#include <stdio.h>
#include <fstream>
//#include "../common/cpu_bitmap.h"


#define BLOCK_SIZE 16

const int N = 100;
__global__ void kernel(float * a, float * b, int n, float * c) //выполняется на устройстве
{
	int bx = blockIdx.x, by = blockIdx.y;
	int tx = threadIdx.x, ty = threadIdx.y;
	int aBegin = n * BLOCK_SIZE * by;
	int aEnd = aBegin + n - 1;
	int bBegin = BLOCK_SIZE * bx;
	int aStep = BLOCK_SIZE, bStep = BLOCK_SIZE * n;
	float sum=0.0f;
	for (int ia = aBegin, ib = bBegin; ia <= aEnd; ia += aStep, ib += bStep) {
		__shared__ float as[BLOCK_SIZE][BLOCK_SIZE];
		__shared__ float bs[BLOCK_SIZE][BLOCK_SIZE];
		as[ty][tx] = a[ia + n * ty + tx];
		bs[ty][tx] = b[ib + n * ty + tx];

		__syncthreads();

		for (int k = 0; k < BLOCK_SIZE; k++)
			sum += as[ty][k] * bs[k][tx];
		__syncthreads();
	}
	c[n * BLOCK_SIZE * by + BLOCK_SIZE * bx + n * ty + tx] = sum;

}


void CPU_keernel(float *a, float *b, int N, float *c) //выполняется на хосте
{
	for (int i = 0; i < N; i++)
	{
		for (int j = 0; j < N; j++)
		{
			c[i*N + j] = 0;
			for (int k = 0; k < N; k++)
			{
				c[i*N + j] += a[i*N + k] * b[k*N + j];
			}
		}
	}
}

void addCPU(int *a, int *b, int *c)
{
    for(int i = 0;i<N;i++)
    {
        c[i] = a[i]+b[i];
    }
        
}

__global__ void add(int *a, int *b, int *c)
{
    int tid = blockIdx.x;// индекс блока ()
    if(tid<N)
    {
        c[tid] = a[tid]+b[tid];
    }
   
}

//****************************для лабы *************************** 

const int Count = 65532;                           //кол-во выводимых чисел

__global__ void printMessagdeGPU()          //Функция выполняема на GPU
{
    int Value = blockIdx.x;
    if(Value <= Count)
    {
        printf("ValueGPU = %d;\n", Value);
    }
    
}

void printMessageCPU()                      //Функция выполняемая на CPU
{
    for(int i=0;i<Count; i++)
    {
        printf("ValueCPU = %d;\n", i);
    }
}


int main(void)
{
	cudaEvent_t start, stop;
	float gpu_time = 0.0;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0);
 
    printMessagdeGPU<<< Count , 1 >>>();
 
    cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&gpu_time, start, stop);
	
	cudaEventDestroy(start);
	cudaEventDestroy(stop);

    cudaEvent_t start1, stop1;
    float cpu_time = 0.0;
    cudaEventCreate(&start1);
	cudaEventCreate(&stop1);
	cudaEventRecord(start1, 0);
    
    printMessageCPU();

    cudaEventRecord(stop1, 0);
	cudaEventSynchronize(stop1);
	cudaEventElapsedTime(&cpu_time, start1, stop1);
	printf("time on cpu = %2fmiliseconds \t time on gpu = %2fmiliseconds  \n\n", cpu_time, gpu_time);
	cudaEventDestroy(start1);
	cudaEventDestroy(stop1);




	return 0;
}





Выходные данные были обрезаны до нескольких последних строк (5000).
ValueCPU = 160535;
ValueCPU = 160536;
ValueCPU = 160537;
ValueCPU = 160538;
ValueCPU = 160539;
ValueCPU = 160540;
ValueCPU = 160541;
ValueCPU = 160542;
ValueCPU = 160543;
ValueCPU = 160544;
ValueCPU = 160545;
ValueCPU = 160546;
ValueCPU = 160547;
ValueCPU = 160548;
ValueCPU = 160549;
ValueCPU = 160550;
ValueCPU = 160551;
ValueCPU = 160552;
ValueCPU = 160553;
ValueCPU = 160554;
ValueCPU = 160555;
ValueCPU = 160556;
ValueCPU = 160557;
ValueCPU = 160558;
ValueCPU = 160559;
ValueCPU = 160560;
ValueCPU = 160561;
ValueCPU = 160562;
ValueCPU = 160563;
ValueCPU = 160564;
ValueCPU = 160565;
ValueCPU = 160566;
ValueCPU = 160567;
ValueCPU = 160568;
ValueCPU = 160569;
ValueCPU = 160570;
ValueCPU = 160571;
ValueCPU = 160572;
ValueCPU = 160573;
ValueCPU = 160574;
ValueCPU = 160575;
ValueCPU = 160576;
ValueCPU = 160577;
ValueCPU = 160578;
ValueCPU = 160579;
ValueCPU = 160580;
ValueCPU = 160581;
ValueCPU = 160582;
ValueCPU = 160583;
V